In [4]:
# Setup Logging
import logging
logging_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO,filename=None, format=logging_format)
log = logging.getLogger(__name__)

from minimal_bert import preproc as pp
from minimal_bert import modelling as mod
from minimal_bert.utils import prepare_and_save_bert_encoded_df as prep_bert
from minimal_bert.utils import load_bert_and_prep_for_training as load_bert

In [ ]:
prep_bert(maxlen=300)

2019-06-10 09:28:58,587 - INFO - Raw data loaded and prepared for processing
2019-06-10 09:28:58,724 - INFO - Encoding sentences using BERT, this will take several minutes


In [ ]:
# Create Parameter Grid
n_sample_li = [100, 500, 1000, 5000, 10000, None]
n_hidden = [16, 32, 48, 64, 128, 256]
param_grid = {'n_hidden': n_hidden, 'number_of_texts': n_sample_li}
param_grid = list(ParameterGrid(param_grid))

for grid in param_grid:
    # Get Data 
    X_train, X_test, y_train, y_test = load_bert(param_grid['number_of_texts'])
    # Train Model
    model, history = mod.train_and_return_model(
        X_train, 
        X_test, 
        y_train, 
        y_test, 
        param_grid['n_hidden'])
    # Extract Results
    break